# Energy community forecasting - Final Project 

> Group made of Ivan Duvnjak, Enkh-Oyu Nomin, Oleg Lastocichin

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch

# import useful libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import kpss


import plotly.express as px
import statsmodels.tsa.stattools
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.graphics.api as smg
from sklearn.metrics import mean_absolute_percentage_error as mape

from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pmdarima as pm
from pmdarima import model_selection

# Probabilistic forecasts with neural models and quantile loss
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.models import LSTM, DilatedRNN, NHITS
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Exploratory analysis

In [ ]:
def read_multiple_pkl_files(file_paths):
    data_list = []
    for path in file_paths:
        try:
            with open(path, 'rb') as file:
                data = pd.read_pickle(file) # use pandas to read pickle files 
                data_list.append(data)
        except Exception as e:
            print(f"Failed to read {path}: {e}")
    return data_list

In [ ]:
files = ['lic_meteo', 'lic_meters', 'lic_nwp']
datas = read_multiple_pkl_files(files)

In [ ]:
df_meteo = datas[0]
df_meteo

In [ ]:
df_meters = datas[1]
df_meters

In [ ]:
df_nwp = datas[2]
df_nwp

In [ ]:
def check_data(df):
    print(df.shape)
    print("Null values: ", df.isna().sum().sum())
    print("Duplicated values: ", df.duplicated().sum().sum())

In [ ]:
for i, data in enumerate(datas):
    print(f"Dataset {i}: ")
    check_data(data)
    print()

As we can see, in our main dataset (df_meters) we have only few missing values regarding the total amount of data available and no duplicated values. Therefore, we will manage later the missing values accordingly or eventually manage them through the parameter of the models (missing='drop').

In [ ]:
df_meteo.describe()

In [ ]:
df_meters.describe()

In [ ]:
df_nwp.describe()

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(15, 15))
axs = axs.ravel()

for i in range(20):
    axs[i].plot(df_meters[i]["e_pos"], color='green', alpha=0.5, label='e_pos')
    axs[i].plot(df_meters[i]["e_neg"], color='red', alpha=0.5, label='e_neg')
    axs[i].set_title(f"House {i+1}")
    axs[i].legend() 

plt.tight_layout() 
plt.show()

We see that not all houses have values for n_neg, most of them don't. Those that have some values, some follow some seasonality 

In [ ]:
# plot values of e_pos & e_neg for "PCC" and "battery"

fig, axs = plt.subplots(nrows=2, figsize=(16, 6))

axs[0].plot(df_meters["PCC"]["e_pos"], color='green', label='e_pos')
axs[0].plot(df_meters["PCC"]["e_neg"], color='red', label='e_neg')
axs[0].set_title('e_neg & e_pos values for PCC') 
axs[0].legend()

axs[1].plot(df_meters["battery"]["e_pos"], color='green', label='e_pos')
axs[1].plot(df_meters["battery"]["e_neg"], color='red', label='e_neg')
axs[1].set_title('e_neg & e_pos values for Battery')  
axs[1].legend()

plt.tight_layout()
plt.show()

We can confirm from the previous plots that e_neg & e_pos are not mutually exclusive. Moreover, most households have e_neg = 0 during the whole time series.

**PCC**: There appears to be a cyclical pattern in the data, possibly indicating seasonal variations in energy usage or production. For example, the peaks in energy seem to repeat annually. The red bars which represent "e_neg" are significantly smaller and less frequent compared to the positive values. This indicates that instances of negative energy (energy injected back to the grid) are much less common.

**Battery**: Unlike the PCC, the battery system shows significant fluctuations between positive and negative energy values. The "e_neg" values, particularly, are much more prominent here than in the PCC plot. There are several sharp spikes in both "e_pos" and "e_neg", which could indicate charging and discharging cycles of the battery. These spikes could also reflect responses to demand shifts or operational adjustments.

### Data Preparation - Hourly utilization

In [ ]:
# create utilization for all houses, battery, pcc and store as columns in dataframe
pdf_meters = pd.DataFrame()

for ind in df_meters.columns.levels[0].to_list():
    df = df_meters[ind]

    df = df.resample('H').mean().dropna()
    df['e_utilization'+ "_" + str(ind)] = df['e_pos'] + df['e_neg'] #define utilization
    df = df.drop(['e_pos','e_neg'],axis=1)

    pdf_meters = pd.concat([pdf_meters,df],axis=1)

pdf_meters

In [ ]:
fig = px.line(pdf_meters, title='PCC e_utilization')
fig

### Check for Lags which have highest correlation

In [ ]:
# Compute Auto correlation 
acf_df = pd.DataFrame()
for cl in pdf_meters.columns:
    acf = pd.DataFrame()
    acf[cl] = statsmodels.tsa.stattools.acf(pdf_meters[cl], missing="drop", nlags=24) # Auto correlation function
    acf_df = pd.concat([acf_df, acf], axis=1)
acf_df

In [ ]:
fig = px.line(acf_df, title='ACF e_utilization')
fig

In [ ]:
top5lags = acf_df.apply(lambda x: pd.Series(x.nlargest(5).index), axis=0)
top5lags 

 We confirm from above auto-correlation analysis that lags 1 and 24 are the most important. This confirms common understanding that energy usage is deterministic from last hour's usage and last day's from same time. 

### Understand relationship between houses and PCC

In [ ]:
# Correlation across series
names = pdf_meters.columns.values
corr_matrix = np.corrcoef(pdf_meters.T)
fig = smg.plot_corr(corr_matrix,xnames=names)
# Overall there exist moderate correlation between different series

#### Test if series are stationary
https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html

**ADF test** - used to determine the presence of unit root in the series, and hence helps in understand if the series is stationary or not

**H0**: The series has a unit root.

**H1**: The series has no unit root.

If the null hypothesis is failed to be rejected, this test may provide evidence that the series is non-stationary.

In [ ]:
# function to return me the non-stationary features by using ADF

def check_stationarity_adf(dataframe):
    non_stationary_columns = []  # List to hold names of non-stationary columns

    for column in dataframe.columns:
        dftest = adfuller(dataframe[column].dropna(), autolag='AIC')  # Dropping NA values to avoid errors

        # Check if the p-value indicates non-stationarity
        if dftest[1] > 0.05:  # p-value greater than 0.05 suggests non-stationarity
            non_stationary_columns.append(column)

    # Print the results after checking all columns
    if non_stationary_columns:
        print(f'Non-stationary columns: {", ".join(non_stationary_columns)}')
    else:
        print("All features are stationary.")

In [ ]:
check_stationarity_adf(pdf_meters)

KPSS is another test for checking the stationarity of a time series. The null and alternate hypothesis for the KPSS test are opposite that of the ADF test.

**KPSS test**

**H0**: The process is trend stationary.

**H1**: The series has a unit root (series is not stationary).

In [ ]:
# function to return me the non-stationary features by using KPSS

def kpss_test(dataframe, **kw):
    non_stationary_columns = []  # List to store non-stationary columns

    # Loop through each column in the DataFrame
    for column in dataframe.columns:
        series = dataframe[column].dropna()  # Drop NA values to avoid errors
        if len(series) == 0:
            print(f"Column {column} only contains NaNs")
            continue
        
        try:
            statistic, p_value, n_lags, critical_values = kpss(series, **kw)
            print(f'KPSS Test Results for {column}:')
            print(f'KPSS Statistic: {statistic}')
            print(f'p-value: {p_value}')
            print(f'num lags: {n_lags}')
            print('Critical Values:')
            for key, value in critical_values.items():
                print(f'    {key} : {value}')
            
            # Determine if the series is non-stationary based on the p-value
            if p_value < 0.05:
                non_stationary_columns.append(column)
                print(f'Result: The series {column} is not stationary\n')
            else:
                print(f'Result: The series {column} is stationary\n')

        except ValueError as e:  # Catch errors related to insufficient data
            print(f"Error testing {column}: {e}")

    # Print all non-stationary columns at the end of the function execution
    if non_stationary_columns:
        print(f'Non-stationary columns: {", ".join(non_stationary_columns)}')
    else:
        print("All columns are stationary.")

In [ ]:
kpss_test(pdf_meters)

KPSS indicates non-stationarity and ADF indicates stationarity - The series is difference stationary. Differencing is to be used to make series stationary. The differenced series is checked for stationarity.

In [ ]:
# def make_stationary_differencing(dataframe):
#     stationary_df = pd.DataFrame()
#     for column in dataframe.columns:
#         stationary_df[column] = dataframe[column] - dataframe[column].shift(1)

#     # dropping the first row which will always be NaN due to differencing
#     stationary_df = stationary_df.dropna()

#     return stationary_df

# stationary_meters = make_stationary_differencing(pdf_meters)

In [ ]:
# # an example of a feature to verify the result
# stationary_meters["e_utilization_PCC"].plot(figsize=(12, 8))
# plt.title("Differentiated e_utilization_PCC")
# plt.show()

In [ ]:
# # an example of a feature to verify the result
# stationary_meters["e_utilization_0"].plot(figsize=(12, 8))
# plt.title("Differentiated e_utilization_0")
# plt.show()

In [ ]:
# pdf_meters["e_utilization_PCC"] = pdf_meters["e_utilization_PCC"] - pdf_meters["e_utilization_PCC"].shift(1)
# pdf_meters["e_utilization_PCC"].dropna().plot(figsize=(12, 8))

In [ ]:
# check_stationarity_adf(stationary_meters) # Check again stationarity after differencing

In [ ]:
# kpss_test(stationary_meters)

In [ ]:
# check_stationarity_adf(pdf_meters["e_utilization_PCC"].dropna())

In [ ]:
# kpss_test(pdf_meters["e_utilization_PCC"].dropna())

As you can see above, the series is difference stationary and we have tried to difference them in order to check if the models improved. We got little to no improvements in the models' performances. Therefore, we left the models that we have performed initially that manage the differencing within the model as you will see below.

## Model benchmark

In [ ]:
losses_mse_all = {}
losses_mae_all = {}

#### Naive method - take the last value as forecast

In [ ]:
nv_fit = pd.DataFrame()
nv_fit = pdf_meters.shift(1) # Set the prediction as previous values 
nv_fit.columns = "Fitted_" + nv_fit.columns
nv_fit = pd.concat([pdf_meters, nv_fit],axis=1)
nv_fit = nv_fit.dropna()
nv_fit

In [ ]:
fig = px.line(nv_fit, title='Fitted-Actual Naive')
fig

In [ ]:
# Use MSE to understand model performance
loss_mae_naive = []
loss_mse_naive = []

for i, cl in enumerate(pdf_meters.columns):
    naive_mse = pd.DataFrame()
    pred_cl = "Fitted_" + str(cl)
    mse_value = mean_squared_error(nv_fit[[cl]], nv_fit[[pred_cl]])  # Calculate MSE once and use it
    mae_value = mean_absolute_error(nv_fit[[cl]], nv_fit[[pred_cl]])
    
    print(cl)
    print(f"MSE: {mse_value.round(3)}")
    print(f"MAE: {mae_value.round(3)}")
    print()

    loss_mae_naive.append(mae_value)
    loss_mse_naive.append(mse_value)

losses_mae_all["naive"] = loss_mae_naive
losses_mse_all["naive"] = loss_mse_naive

#### Exponential smoothing with Holt-Winters

In [ ]:
hw_fit = pd.DataFrame()

for cl in pdf_meters.columns:
    res = pd.DataFrame()
    model = ExponentialSmoothing(pdf_meters[cl], trend="add", seasonal="add", seasonal_periods=24).fit() # use exponential smoothing with 24 hours
    print(model.summary())
    res[cl] = pdf_meters[cl]
    f_res = "Fitted_" + str(cl)

    res[f_res] = model.fittedvalues

    hw_fit = pd.concat([hw_fit,res],axis=1)
    hw_fit = hw_fit.dropna()

In [ ]:
fig = px.line(hw_fit, title='Fitted-Actual Holt Winters')
fig

In [ ]:
loss_mae_hw = []
loss_mse_hw = []

for i, cl in enumerate(pdf_meters.columns):
    pred_cl = "Fitted_" + str(cl)
    
    mse_value = mean_squared_error(hw_fit[[cl]] , hw_fit[[pred_cl]])  # Calculate MSE once and use it
    mae_value = mean_absolute_error(hw_fit[[cl]] , hw_fit[[pred_cl]])
    
    print(cl)
    print(f"MSE: {mse_value.round(3)}")
    print(f"MAE: {mae_value.round(3)}")
    print()

    loss_mae_hw.append(mae_value)
    loss_mse_hw.append(mse_value)

losses_mae_all["hw"] = loss_mae_hw
losses_mse_all["hw"] = loss_mse_hw

#### AR-1 : Auto-regressive lag 1, 24 model

In [ ]:
# AR 1,24 model
ar1_fit = pd.DataFrame()

for cl in pdf_meters.columns:
    res = pd.DataFrame()
    model = AutoReg(pdf_meters[cl], lags = [1,24], missing="drop").fit()
    print(model.summary())
    res[cl] = pdf_meters[cl]
    f_res = "Fitted_" + str(cl)

    res[f_res] = model.fittedvalues


    ar1_fit = pd.concat([ar1_fit,res],axis=1)
    ar1_fit = ar1_fit.dropna()


In [ ]:
fig = px.line(ar1_fit, title='Fitted-Actual AutoRegressive lags 1,24')
fig

#### Log Model metrics

In [ ]:
#Use MsE to understand model performance
loss_mae_ar1 = []
loss_mse_ar1 = []

for i, cl in enumerate(pdf_meters.columns):
    pred_cl = "Fitted_" + str(cl)
    
    mse_value = mean_squared_error(ar1_fit[[cl]] , ar1_fit[[pred_cl]])  # Calculate MSE once and use it
    mae_value = mean_absolute_error(ar1_fit[[cl]] , ar1_fit[[pred_cl]])
    
    print(cl)
    print(f"MSE: {mse_value.round(3)}")
    print(f"MAE: {mae_value.round(3)}")
    print()

    loss_mae_ar1.append(mae_value)
    loss_mse_ar1.append(mse_value)

losses_mae_all["ar1"] = loss_mae_ar1
losses_mse_all["ar1"] = loss_mse_ar1

## Model proposal and training

#### Fit ARIMA model with differencing to make the series stationary in trend and see if the performance improves

In [ ]:
# ARIMA model
arima_fit = pd.DataFrame()

for cl in pdf_meters.columns:
    res = pd.DataFrame()
    model = ARIMA(pdf_meters[cl], order=(1,1,1), missing="drop").fit()
    print(model.summary())
    res[cl] = pdf_meters[cl]
    f_res = "Fitted_" + str(cl)

    res[f_res] = model.fittedvalues


    arima_fit = pd.concat([arima_fit,res],axis=1)
    arima_fit = arima_fit.dropna()

In [ ]:
fig = px.line(arima_fit, title='Fitted-Actual ARIMA')
fig

In [ ]:
#Use MsE to understand model performance
loss_mae_arima_fit = []
loss_mse_arima_fit = []

for i, cl in enumerate(pdf_meters.columns):
    pred_cl = "Fitted_" + str(cl)
    
    mse_value = mean_squared_error(arima_fit[[cl]] , arima_fit[[pred_cl]])  # Calculate MSE once and use it
    mae_value = mean_absolute_error(arima_fit[[cl]] , arima_fit[[pred_cl]])
    
    print(cl)
    print(f"MSE: {mse_value.round(3)}")
    print(f"MAE: {mae_value.round(3)}")
    print()

    loss_mae_arima_fit.append(mae_value)
    loss_mse_arima_fit.append(mse_value)

losses_mae_all["arima_fit"] = loss_mae_arima_fit
losses_mse_all["arima_fit"] = loss_mse_arima_fit

#### Moving average model without differencing

In [ ]:
# MA model
ma_fit = pd.DataFrame()

for cl in pdf_meters.columns:
    res = pd.DataFrame()
    model = ARIMA(pdf_meters[cl], order=(0,0,1)).fit()
    print(model.summary())
    res[cl] = pdf_meters[cl]
    f_res = "Fitted_" + str(cl)

    res[f_res] = model.fittedvalues


    ma_fit = pd.concat([ma_fit,res],axis=1)
    ma_fit = ma_fit.dropna()

In [ ]:
fig = px.line(ma_fit, title='Fitted-Actual')
fig

In [ ]:
#Use MsE to understand model performance
loss_mae_ma_fit = []
loss_mse_ma_fit = []

for i, cl in enumerate(pdf_meters.columns):
    pred_cl = "Fitted_" + str(cl)
    
    mse_value = mean_squared_error(ma_fit[[cl]] , ma_fit[[pred_cl]])  # Calculate MSE once and use it
    mae_value = mean_absolute_error(ma_fit[[cl]] , ma_fit[[pred_cl]])
    
    print(cl)
    print(f"MSE: {mse_value.round(3)}")
    print(f"MAE: {mae_value.round(3)}")
    print()

    loss_mae_ma_fit.append(mae_value)
    loss_mse_ma_fit.append(mse_value)

losses_mae_all["ma_fit"] = loss_mae_ma_fit
losses_mse_all["ma_fit"] = loss_mse_ma_fit

In [ ]:
benchmark_mae = pd.DataFrame(pd.DataFrame(losses_mae_all).T)
benchmark_mae.columns = pdf_meters.columns

benchmark_mse = pd.DataFrame(pd.DataFrame(losses_mse_all).T)
benchmark_mse.columns = pdf_meters.columns

In [ ]:
fig = px.line(benchmark_mae, title='MAE benchmark')
fig

In [ ]:
fig = px.line(benchmark_mse, title="MSE benchmark")
fig

In [ ]:
benchmark_mse.T.describe()

In [ ]:
benchmark_mae.T.describe()

From the plots we see that the error is more or less the same across all models when excluding the PCC, that is really high since all its values are also really high. 

By looking at the mean value of the table we see that AR1 has the lowest values across the models, both mse and mae.

## Model selection

#### Overall, AR1 model performs well as compared to above models, still lets experiment different ARIMA model ( AR , MA, ARMA) and test model performance using CV

https://alkaline-ml.com/pmdarima/auto_examples/model_selection/example_cross_validation.html

In [ ]:
losses_mae_columns = []
losses_mse_columns = []

for cl in pdf_meters.columns:
    print("Model for: " + cl)
    train, test = model_selection.train_test_split(pdf_meters[cl], test_size=24)
    
    arma = pm.ARIMA(order=(1, 0, 1),
                    seasonal_order=(0, 1, 1, 24), # choose 24 for daily seasonality, seen in auto regressive plots
                    suppress_warnings=True)
    

    window_size = np.floor(len(train) / 40) # making the window a bit shorter to save time 
    step = np.floor(len(train) / 40)
    cv = model_selection.SlidingWindowForecastCV(window_size=int(window_size), step=int(step), h=24) 
    splitter_cv = cv.split(train)

    losses_mae = []
    losses_mse = []

    for i in range(5): # 5 folds 
        tr_idx, te_idx = next(splitter_cv)
        x_tr = train[tr_idx]
        y_tr = train[te_idx]


        arma.fit(x_tr)
        pred = arma.predict(len(y_tr))
        mse = mean_squared_error(y_tr, pred)
        mae = mean_absolute_error(y_tr, pred)
        losses_mae.append(mae)
        losses_mse.append(mse)
        print("MSE ", mse)
        print("MAE ", mae)

        plt.plot(x_tr[-128:], c='blue', label="Train")
        plt.plot(pred[:24], c="green", label="Forecast")
        plt.plot(y_tr[:24], c="red", label="Test")
        plt.legend()
        plt.show()
    
    losses_mae_columns.append(losses_mae)
    losses_mse_columns.append(losses_mse)

The plots indicate that the model is generally capable of making accurate forecasts.

In [ ]:
mae_errors = pd.DataFrame(np.array(losses_mae_columns).T, columns=pdf_meters.columns)
mse_errors = pd.DataFrame(np.array(losses_mse_columns).T, columns=pdf_meters.columns)

In [ ]:
fig = px.line(mae_errors, title='MAE')
fig

In [ ]:
fig = px.line(mse_errors, title='MSE')
fig

In [ ]:
mae_errors.describe()

In [ ]:
mse_errors.describe()

We see that the model has a stable performance across the different splits most of the houses.  
While for the PCC forecast struggles a bit.

## Probabilistic forecast model using LSTM and NHITS models

We defined a preprocessing function to prepare the data to be used in a NeuralForecast model, the data had to be defined in a predefined preset that included:
- ds = the datetime
- y = the target values
- unique_id = the column name in our case the house or pcc

As such we proceeded to transform the data to satisfy this preset.
We converted the datetimes to not include the timezone value.
Interpolated missing values, there is no option to drop them like previously used.


In [ ]:
 #function to preprocess each column for the specific models
def preprocess_column(column_name, column_data, test_size):
    df_lstm = pd.DataFrame(column_data)

    df_lstm.rename(columns={column_name: 'y'}, inplace=True) # define target

    df_lstm.reset_index(inplace=True)
    df_lstm['ds'] = pd.to_datetime(df_lstm['index']) # move the datetimes from index to the column ds
    df_lstm.drop(columns=['index'], inplace=True)
    # transformations to ds to comply to the needed datatype
    df_lstm['ds'] = df_lstm['ds'].dt.tz_localize(None)
    df_lstm['ds'] = df_lstm['ds'].astype('datetime64[ns]')
    # assigning unique id value for the column pre-processed
    df_lstm["unique_id"] = column_name    

    # interpolate missing values
    df_lstm['y'] = df_lstm['y'].interpolate(method='linear', limit_direction='both')
    # splitting
    df_tr, df_te = model_selection.train_test_split(df_lstm, test_size=test_size)
    return df_tr, df_te

Function to create our models, this will be used for all the horizons we decided to work on, which are: 24, 12, 1.

In [ ]:
def create_model(horizon, levels):
    models = [LSTM(input_size=-1, h=horizon,
                loss=MQLoss(level=levels), max_steps=1000),
            NHITS(input_size=7*horizon, h=horizon,
                    n_freq_downsample=[24, 12, 1],
                    loss=MQLoss(level=levels), max_steps=250),]
    fcst = NeuralForecast(models=models, freq="H")
    return fcst

Simple function to save computing time and actual time, it loads or save the models previously trained.

In [ ]:
def save_or_load_model(model_path, fcst, df_tr):
    if os.path.exists(model_path):
        fcst = fcst.load(model_path)
    else:
        fcst.fit(df=df_tr)
        fcst.save(model_path)
    return fcst

NMAE calculator

In [ ]:
nmae = lambda x,y: np.mean(np.abs(x-y))/(np.mean(x)+1e-6) # nmae used by the teacher

Print NMAE and MSE given the name of the model, training dataset, forecasts, test dataset

In [ ]:
def print_measures(model, forecasts, df_tr, df_te):
  for uid in df_tr['unique_id'].unique():
      df_test = df_te.reset_index().drop("index", axis=1)
      y_te = df_test.loc[df_test['unique_id'] == uid, 'y']
      y_hat = forecasts.loc[forecasts['unique_id'] == uid, f'{model}-median']
      print(f'{uid} NMAE:{nmae(y_te.values.ravel(), y_hat.values.ravel()):0.2e}, MSE:{mean_squared_error(y_te.values.ravel(), y_hat.values.ravel()):0.2e}')


Function to plot the results, using the same values as before

In [ ]:
def plot_results(model, forecasts, df_tr, df_te):
  q_names = ['lo-90','lo-80','hi-90','hi-80']

  fig, ax = plt.subplots(3, 1, figsize=(18, 12))

  plt.subplots_adjust(hspace=0.25)
  for uid, a in zip(df_tr['unique_id'].unique(), ax.ravel()):
      df_test = df_te.reset_index().drop("index", axis=1)
      y_te = df_test.loc[df_test['unique_id'] == uid, 'y']

      y_hat = forecasts.loc[forecasts['unique_id'] == uid, '{}-median'.format(model)]
      
      y_te.plot(ax=a)
      [a.plot(forecasts.loc[forecasts['unique_id'] == uid, '{}-{}'.format(model, q_str)], color='red', alpha=0.2) for q_str in q_names]
      y_hat.plot(ax=a)
      a.set_title(f'{uid} NMAE:{nmae(y_te.values.ravel(), y_hat.values.ravel()):0.2e}, MSE:{mean_squared_error(y_te.values.ravel(), y_hat.values.ravel()):0.2e}')
      a.legend()
  plt.suptitle(model)

Our first models, that have an horizon of 24

In [ ]:
dfs_tr = []
dfs_te = []
for column_name in pdf_meters.columns:
    if column_name == 'unique_id':
        continue
    if column_name in ("e_utilization_3", "e_utilization_6", "e_utilization_PCC"):
        df_tr, df_te = preprocess_column(column_name, pdf_meters[column_name], 24)
        dfs_tr.append(df_tr)
        dfs_te.append(df_te)

# Concatenate all preprocessed DataFrames
df_tr = pd.concat(dfs_tr)
df_te = pd.concat(dfs_te)

Defining the model using the functions from before, in this case it will be for a 24 step model and will be saved as suck.

In [ ]:
horizon = 24 # 24 steps, 1 day prediction
levels = [80, 90]
base_model_24 = create_model(horizon, levels)

model_path_24 = 'neural_forecaster_model_24'
fcst_24 = save_or_load_model(model_path_24, base_model_24, df_tr)

Using the model to forecast

In [ ]:
forecasts_24 = fcst_24.predict()
forecasts_24 = forecasts_24.reset_index()
forecasts_24.head()

Plotting and print the results

In [ ]:
print_measures('LSTM', forecasts_24, df_tr, df_te)
print()
print_measures('NHITS', forecasts_24, df_tr, df_te)

plot_results('LSTM', forecasts_24, df_tr, df_te)
plot_results('NHITS', forecasts_24, df_tr, df_te)


The forecasted median does follow well the true values for every column.

Applying the same idea to a different horizon and see how the results change

In [ ]:
dfs_tr = []
dfs_te = []

for column_name in pdf_meters.columns:
    if column_name == 'unique_id':
        continue
    if column_name in ("e_utilization_3", "e_utilization_6", "e_utilization_PCC"):
        df_tr, df_te = preprocess_column(column_name, pdf_meters[column_name], 12) # horizon 12
        dfs_tr.append(df_tr)
        dfs_te.append(df_te)

# Concatenate all preprocessed DataFrames
df_tr = pd.concat(dfs_tr)
df_te = pd.concat(dfs_te)

In [ ]:
horizon = 12 # 12 steps, half day prediction
levels = [80, 90]
base_model_12 = create_model(horizon, levels)

# Check if model file exists
model_path_12 = 'neural_forecaster_model_12'

fcst_12 = save_or_load_model(model_path_12, base_model_12, df_tr)

In [ ]:
# Now you can use fcst for inference
forecasts_12 = fcst_12.predict()
forecasts_12 = forecasts_12.reset_index()
forecasts_12.head()

In [ ]:
df_te.reset_index().drop("index", axis=1)

print_measures('LSTM', forecasts_12, df_tr, df_te)
print()
print_measures('NHITS', forecasts_12, df_tr, df_te)


plot_results('LSTM', forecasts_12, df_tr, df_te)
plot_results('NHITS', forecasts_12, df_tr, df_te)

Similarly to the plot above the prediction of the median looks good

As before we will apply the same concepts but with a narrowed horizon of 1

In [ ]:
dfs_tr = []
dfs_te = []
for column_name in pdf_meters.columns:
    if column_name == 'unique_id':
        continue
    if column_name in ("e_utilization_3", "e_utilization_6", "e_utilization_PCC"):
        df_tr, df_te = preprocess_column(column_name, pdf_meters[column_name], 1)
        dfs_tr.append(df_tr)
        dfs_te.append(df_te)
    
df_tr = pd.concat(dfs_tr)
df_te = pd.concat(dfs_te)

In [ ]:

model_path_1 = 'neural_forecaster_model_1'
horizon = 1 # 1 steps, 1 hour prediction
levels = [80, 90]
base_model_1 = create_model(horizon, levels)

fcst_1 = save_or_load_model(model_path_1, base_model_1, df_tr)


In [ ]:
# use fcst for inference
forecasts_1 = fcst_1.predict()
forecasts_1 = forecasts_1.reset_index()
forecasts_1.head()

In [ ]:
df_te.reset_index().drop("index", axis=1)

In [ ]:
print_measures("LSTM", forecasts_1, df_tr, df_te)
print()
print_measures("NHITS", forecasts_1, df_tr, df_te)